In [1]:
import sys
import os
sys.path.append('../dags')

from database import create_database_engine


# 환경 변수 설정
os.environ["POSTGRES_USER"] = "airflow"
os.environ["POSTGRES_PASSWORD"] = "airflow"
os.environ["POSTGRES_DB"] = "events"
os.environ["POSTGRES_PORT"] = "5433"

engine = create_database_engine(host="127.0.0.1")
print("Successfully connected to the database")

Successfully connected to the database


In [2]:
import pandas as pd
from datetime import datetime

# 나머지는 변수로 관리
label_table = "kind"
return_table = "abnormal_return_kind"
label_col = "label"

abn_return_cols = [f"abn_ret_{m}m" for m in range(1, 31)]
volume_cols = [
    "cum_volume_30m", "cum_volume_minus_30m", 
]

# SQL 컬럼 문자열을 만듬 (event_id, event_ts는 하드코딩)
sql_columns = (
    [f"ar.event_ts"] +
    [f"ar.{col}" for col in abn_return_cols] +
    [f"ar.{col}" for col in volume_cols] +
    [f"k.{label_col}"]
)
sql_columns_str = ",\n    ".join(sql_columns)

query = f"""
SELECT {sql_columns_str}
FROM {return_table} ar
JOIN "{label_table}" k ON ar.event_id = k.id
ORDER BY ar.event_ts ASC
"""

df_total = pd.read_sql(query, engine)
df_total["telegram_dummy"] = (
    pd.to_datetime(df_total["event_ts"], utc=True) > pd.Timestamp("2022-06-30", tz="UTC")
).astype(int)
df_total

DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.41.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE events REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


,event_ts,abn_ret_1m,abn_ret_2m,abn_ret_3m,abn_ret_4m,abn_ret_5m,abn_ret_6m,abn_ret_7m,abn_ret_8m,abn_ret_9m,...,abn_ret_25m,abn_ret_26m,abn_ret_27m,abn_ret_28m,abn_ret_29m,abn_ret_30m,cum_volume_30m,cum_volume_minus_30m,label,telegram_dummy
0,2021-01-04 10:08:00+00:00,0.75,-0.35,-0.18,0.19,-0.20,-0.23,-0.24,-0.67,-0.34,...,-0.33,0.07,0.21,0.08,0.05,0.05,81261.0,37813.0,0,0
1,2021-01-04 10:08:00+00:00,-0.07,-0.32,-2.66,-2.49,-2.27,-1.24,-1.62,-1.37,-1.04,...,-0.76,-1.24,-1.69,-1.43,-1.47,-1.47,88710.0,34208.0,1,0
2,2021-01-04 10:08:00+00:00,0.75,-0.35,-0.18,0.19,-0.20,-0.23,-0.24,-0.67,-0.34,...,-0.33,0.07,0.21,0.08,0.05,0.05,81261.0,37813.0,0,0
3,2021-01-04 10:14:00+00:00,0.10,-0.13,-2.60,-4.00,-3.64,-3.51,-3.64,-4.17,-4.70,...,-4.26,-3.20,-3.50,-2.30,-3.46,-3.46,1195841.0,200844.0,1,0
4,2021-01-04 10:16:00+00:00,0.46,0.47,0.25,0.19,0.06,-0.24,-0.06,-0.24,-0.50,...,-0.81,-0.68,-0.67,-0.75,-0.52,-0.54,167284.0,209721.0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8651,2023-12-28 14:46:00+00:00,3.30,9.04,9.00,13.65,7.01,7.60,7.93,7.26,7.78,...,6.35,6.17,5.98,5.76,5.70,5.17,152308.0,5681.0,1,1
8652,2023-12-28 14:53:00+00:00,0.67,-0.64,0.24,0.29,0.21,0.21,0.23,0.27,0.27,...,-0.24,-0.29,-0.29,NaN,NaN,NaN,34468.0,60477.0,1,1
8653,2023-12-28 14:56:00+00:00,-0.01,0.42,0.42,0.44,0.48,0.48,0.45,0.45,0.42,...,NaN,NaN,NaN,NaN,NaN,NaN,23002.0,7337.0,0,1
8654,2023-12-28 14:56:00+00:00,-0.01,0.42,0.42,0.44,0.48,0.48,0.45,0.45,0.42,...,NaN,NaN,NaN,NaN,NaN,NaN,23002.0,7337.0,0,1


In [3]:
df_total = df_total.dropna()

print(df_total.shape[0])

7436


In [4]:
import numpy as np

for m in [30]:
    denom = 1 + df_total[f"cum_volume_minus_{m}m"]
    num = 1 + df_total[f"cum_volume_{m}m"]
    df_total[f"delta_cum_volume_{m}m"] = np.log(num / denom)

for m in [30]:
    col = f"delta_cum_volume_{m}m"
    x = df_total[col].values
    x_min = np.nanmin(x)
    x_max = np.nanmax(x)
    df_total[col] = 2 * (x - x_min) / (x_max - x_min) - 1

# 누적 거래량으로 시작하는 컬럼 모두 drop
cum_cols_to_drop = [col for col in df_total.columns if col.startswith("cum_volume_")]
df_total = df_total.drop(columns=cum_cols_to_drop)
df_total.filter(regex="^delta_cum_").head()


/var/folders/5r/tdks4c2x5vgdpqkw390mgzqm0000gn/T/ipykernel_45070/2492236170.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total[f"delta_cum_volume_{m}m"] = np.log(num / denom)
/var/folders/5r/tdks4c2x5vgdpqkw390mgzqm0000gn/T/ipykernel_45070/2492236170.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total[col] = 2 * (x - x_min) / (x_max - x_min) - 1


,delta_cum_volume_30m
0,0.042062
1,0.067783
2,0.042062
3,0.181565
4,-0.093610


In [5]:
df_total.filter(regex="^delta_cum_").describe()


,delta_cum_volume_30m
count,7436.000000
mean,0.031621
std,0.145088
min,-1.000000
25%,-0.051840
50%,0.018529
75%,0.105876
max,1.000000


In [6]:
positive_df = df_total[df_total["label"] == 1]
positive_df["time_to_peak"] = positive_df[abn_return_cols].idxmax(axis=1).str.extract(r'(\d+)').astype(int)

negative_df = df_total[df_total["label"] == -1]
negative_df["time_to_peak"] = negative_df[abn_return_cols].idxmin(axis=1).str.extract(r'(\d+)').astype(int)

/var/folders/5r/tdks4c2x5vgdpqkw390mgzqm0000gn/T/ipykernel_45070/2394544948.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_df["time_to_peak"] = positive_df[abn_return_cols].idxmax(axis=1).str.extract(r'(\d+)').astype(int)
/var/folders/5r/tdks4c2x5vgdpqkw390mgzqm0000gn/T/ipykernel_45070/2394544948.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_df["time_to_peak"] = negative_df[abn_return_cols].idxmin(axis=1).str.extract(r'(\d+)').astype(int)


In [7]:
positive_df

,event_ts,abn_ret_1m,abn_ret_2m,abn_ret_3m,abn_ret_4m,abn_ret_5m,abn_ret_6m,abn_ret_7m,abn_ret_8m,abn_ret_9m,...,abn_ret_25m,abn_ret_26m,abn_ret_27m,abn_ret_28m,abn_ret_29m,abn_ret_30m,label,telegram_dummy,delta_cum_volume_30m,time_to_peak
1,2021-01-04 10:08:00+00:00,-0.07,-0.32,-2.66,-2.49,-2.27,-1.24,-1.62,-1.37,-1.04,...,-0.76,-1.24,-1.69,-1.43,-1.47,-1.47,1,0,0.067783,1
3,2021-01-04 10:14:00+00:00,0.10,-0.13,-2.60,-4.00,-3.64,-3.51,-3.64,-4.17,-4.70,...,-4.26,-3.20,-3.50,-2.30,-3.46,-3.46,1,0,0.181565,1
13,2021-01-05 10:54:00+00:00,-0.05,-0.02,-0.10,-0.10,-0.19,-0.24,-0.07,-0.11,-0.07,...,-0.05,0.05,0.08,-0.03,-0.07,0.02,1,0,-0.139722,10
15,2021-01-05 11:08:00+00:00,-0.16,0.25,-0.51,-0.51,-0.10,-0.89,-0.95,-0.92,-0.58,...,-0.53,-0.97,-1.00,-1.00,-1.00,-0.91,1,0,-0.120788,2
17,2021-01-05 11:42:00+00:00,0.45,0.04,0.23,0.23,0.42,0.20,-0.06,0.14,0.10,...,0.04,0.02,-0.13,-0.12,-0.15,-0.11,1,0,-0.075333,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8640,2023-12-27 14:33:00+00:00,0.17,-0.17,-0.04,0.04,-0.17,-0.13,-0.09,-0.13,0.07,...,-0.67,-0.87,-0.87,-0.54,-0.50,-0.54,1,1,-0.157332,1
8643,2023-12-28 09:25:00+00:00,0.39,0.38,0.15,0.16,0.13,0.32,0.88,1.31,1.29,...,1.16,1.10,1.51,1.73,1.71,1.95,1,1,0.117760,30
8644,2023-12-28 10:40:00+00:00,0.25,-0.11,-0.46,-1.48,-1.23,-1.74,-1.92,-1.99,-2.18,...,-1.19,-1.23,-0.78,-0.89,-0.92,-1.44,1,1,0.079479,1
8646,2023-12-28 11:10:00+00:00,-4.83,-5.90,-5.93,-6.23,-6.86,-6.75,-7.79,-7.79,-7.61,...,-9.38,-9.81,-9.49,-9.44,-10.17,-10.21,1,1,0.057859,1


In [8]:
negative_df

,event_ts,abn_ret_1m,abn_ret_2m,abn_ret_3m,abn_ret_4m,abn_ret_5m,abn_ret_6m,abn_ret_7m,abn_ret_8m,abn_ret_9m,...,abn_ret_25m,abn_ret_26m,abn_ret_27m,abn_ret_28m,abn_ret_29m,abn_ret_30m,label,telegram_dummy,delta_cum_volume_30m,time_to_peak
4,2021-01-04 10:16:00+00:00,0.46,0.47,0.25,0.19,0.06,-0.24,-0.06,-0.24,-0.50,...,-0.81,-0.68,-0.67,-0.75,-0.52,-0.54,-1,0,-0.093610,20
28,2021-01-07 14:13:00+00:00,-0.07,-0.20,-0.23,-0.23,-0.26,-0.23,0.20,0.23,-0.13,...,-0.76,-0.72,-0.82,-0.86,-0.89,-0.89,-1,0,0.056610,23
157,2021-01-25 10:36:00+00:00,0.00,-0.03,-4.57,-4.22,-4.49,-5.40,-4.98,-3.10,-4.42,...,-4.20,-3.68,-3.36,-3.01,-2.66,-3.36,-1,0,0.199716,6
158,2021-01-25 10:53:00+00:00,0.68,0.38,0.03,0.01,-0.05,0.27,0.18,0.15,-0.23,...,-1.02,-1.08,-0.74,-0.75,-1.18,-0.82,-1,0,-0.106148,29
167,2021-01-25 13:47:00+00:00,0.06,0.13,0.13,0.03,-0.03,0.03,0.10,0.10,0.03,...,-0.19,-0.23,-0.23,-0.16,-0.23,-0.29,-1,0,0.189055,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8431,2023-12-01 10:46:00+00:00,-2.17,-1.81,-1.42,-1.38,-1.27,-1.23,-1.19,-1.12,-1.08,...,-2.25,-2.14,-2.10,-1.98,-2.18,-2.18,-1,1,-0.150940,25
8504,2023-12-12 14:15:00+00:00,0.02,-0.06,-0.10,-0.13,-0.10,-0.02,-0.02,0.02,-0.15,...,0.29,0.25,0.29,0.22,0.22,0.14,-1,1,-0.026658,9
8537,2023-12-15 11:24:00+00:00,-0.76,-0.48,-1.28,-0.83,-0.82,-0.30,-0.58,-0.55,-0.83,...,-0.59,-0.06,-0.58,-0.58,-0.30,-0.30,-1,1,0.034749,3
8579,2023-12-20 14:46:00+00:00,-0.04,-0.06,0.06,-0.04,0.06,-0.01,-0.04,0.03,0.05,...,-0.26,-0.12,-0.07,-0.07,-0.14,-0.02,-1,1,-0.066721,25


In [9]:
positive_df[["time_to_peak", "telegram_dummy"]].groupby("telegram_dummy").describe()


time_to_peak                                                 
                      count       mean        std  min  25%  50%   75%   max
telegram_dummy                                                              
0                    2331.0  11.172029  10.148046  1.0  2.0  7.0  20.0  30.0
1                    1962.0  10.840979   9.943809  1.0  2.0  7.0  19.0  30.0

In [10]:
negative_df[["time_to_peak", "telegram_dummy"]].groupby("telegram_dummy").describe()


time_to_peak                                                  
                      count       mean        std  min  25%   50%   75%   max
telegram_dummy                                                               
0                     362.0  14.856354  10.155376  1.0  5.0  14.0  25.0  30.0
1                     229.0  15.000000  10.078638  1.0  5.0  15.0  25.0  30.0

In [11]:
from hypothesis_test_TTP import regress_ln_ttp_with_volume, regress_ttp_with_volume, regress_ln_ttp, regress_ttp

summary = regress_ln_ttp_with_volume(positive_df, volume_col="delta_cum_volume_30m")
summary = regress_ln_ttp_with_volume(negative_df, volume_col="delta_cum_volume_30m")
df_total_ttp = pd.concat([positive_df, negative_df])
summary = regress_ln_ttp_with_volume(df_total_ttp, volume_col="delta_cum_volume_30m")


--- ln(time_to_peak) 회귀 (delta_cum_volume_30m) ---
            variable       coef    std  t_stat  p_value
               const  2.0777*** 0.0211  98.687 0.000000
      telegram_dummy    -0.0209 0.0304  -0.686 0.492904
delta_cum_volume_30m -0.4672*** 0.1034  -4.519 0.000006
R-squared: 0.0047, Adj R-squared: 0.0042, n_obs: 4293

--- ln(time_to_peak) 회귀 (delta_cum_volume_30m) ---
            variable      coef    std  t_stat  p_value
               const 2.4499*** 0.0474  51.728 0.000000
      telegram_dummy    0.0220 0.0746   0.294 0.768498
delta_cum_volume_30m    0.3056 0.2773   1.102 0.270362
R-squared: 0.0027, Adj R-squared: -0.0007, n_obs: 591

--- ln(time_to_peak) 회귀 (delta_cum_volume_30m) ---
            variable       coef    std  t_stat  p_value
               const  2.1270*** 0.0194 109.546 0.000000
      telegram_dummy    -0.0267 0.0285  -0.938 0.348334
delta_cum_volume_30m -0.4494*** 0.0967  -4.646 0.000003
R-squared: 0.0045, Adj R-squared: 0.0041, n_obs: 4884


In [12]:
summary = regress_ttp_with_volume(positive_df, volume_col="delta_cum_volume_30m")
summary = regress_ttp_with_volume(negative_df, volume_col="delta_cum_volume_30m")
summary = regress_ttp_with_volume(df_total_ttp, volume_col="delta_cum_volume_30m")


--- ln(time_to_peak) 회귀 (10m) ---
            variable       coef    std  t_stat  p_value
               const 11.3484*** 0.2142  52.968 0.000000
      telegram_dummy    -0.3069 0.3070  -1.000 0.317501
delta_cum_volume_30m -4.4649*** 1.0583  -4.219 0.000025
R-squared: 0.0043, Adj R-squared: 0.0039, n_obs: 4293

--- ln(time_to_peak) 회귀 (10m) ---
            variable       coef    std  t_stat  p_value
               const 14.8601*** 0.5346  27.797 0.000000
      telegram_dummy     0.1171 0.8543   0.137 0.890957
delta_cum_volume_30m     2.3641 2.9956   0.789 0.430006
R-squared: 0.0012, Adj R-squared: -0.0022, n_obs: 591

--- ln(time_to_peak) 회귀 (10m) ---
            variable       coef    std  t_stat  p_value
               const 11.8153*** 0.2000  59.069 0.000000
      telegram_dummy    -0.3601 0.2908  -1.238 0.215591
delta_cum_volume_30m -4.3551*** 1.0008  -4.352 0.000014
R-squared: 0.0042, Adj R-squared: 0.0038, n_obs: 4884


In [13]:
summary = regress_ln_ttp(positive_df)
summary = regress_ln_ttp(negative_df)
summary = regress_ln_ttp(df_total_ttp)


--- ln(time_to_peak) 회귀 (no volume) ---
      variable      coef    std  t_stat  p_value
         const 1.8056*** 0.0256  70.619 0.000000
telegram_dummy   -0.0268 0.0377  -0.711 0.476937
R-squared: 0.0001, Adj R-squared: -0.0001, n_obs: 4293

--- ln(time_to_peak) 회귀 (no volume) ---
      variable      coef    std  t_stat  p_value
         const 2.2872*** 0.0569  40.217 0.000000
telegram_dummy    0.0366 0.0889   0.411 0.680811
R-squared: 0.0003, Adj R-squared: -0.0014, n_obs: 591

--- ln(time_to_peak) 회귀 (no volume) ---
      variable      coef    std  t_stat  p_value
         const 1.8703*** 0.0236  79.174 0.000000
telegram_dummy   -0.0346 0.0351  -0.984 0.325301
R-squared: 0.0002, Adj R-squared: -0.0000, n_obs: 4884


In [14]:
summary = regress_ttp(positive_df)
summary = regress_ttp(negative_df)
summary = regress_ttp(df_total_ttp)


--- time_to_peak 회귀 (no volume) ---
      variable       coef    std  t_stat  p_value
         const 11.1720*** 0.2102  53.151 0.000000
telegram_dummy    -0.3311 0.3075  -1.076 0.281716
R-squared: 0.0003, Adj R-squared: 0.0000, n_obs: 4293

--- time_to_peak 회귀 (no volume) ---
      variable       coef    std  t_stat  p_value
         const 14.8564*** 0.5339  27.825 0.000000
telegram_dummy     0.1436 0.8534   0.168 0.866322
R-squared: 0.0000, Adj R-squared: -0.0016, n_obs: 591

--- time_to_peak 회귀 (no volume) ---
      variable       coef    std  t_stat  p_value
         const 11.6673*** 0.1970  59.215 0.000000
telegram_dummy    -0.3916 0.2912  -1.345 0.178681
R-squared: 0.0004, Adj R-squared: 0.0002, n_obs: 4884
